# Transformer

Transformer의 핵심 구조인 Multi-head Attention을 구현하는 실습입니다.
1. Multi-head attention 및 self-attention 구현.
2. 각 과정에서 일어나는 연산과 input/output 형태 이해.

### 필요 패키지 import

In [1]:
from torch import nn
from torch.nn import functional as F
from tqdm import tqdm

import torch
import math

## Req. 2-1 Multi-head self-attention 구조 익히기

### **데이터 전처리**
vocab_size 100인 가상의 시퀀스 데이터를 생성합니다. 

각 데이터에 할당된 숫자는 tokenizing과 정수화가 이뤄진 형태입니다.

In [2]:
pad_id = 0
vocab_size = 100

data = [
  [62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75],
  [60, 96, 51, 32, 90],
  [35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54],
  [75, 51],
  [66, 88, 98, 47],
  [21, 39, 10, 64, 21],
  [98],
  [77, 65, 51, 77, 19, 15, 35, 19, 23, 97, 50, 46, 53, 42, 45, 91, 66, 3, 43, 10],
  [70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34],
  [20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43]
]

In [3]:
# 길이 맞춰주기 위해 패딩합니다.
def padding(data):
  max_len = len(max(data, key=len))
  print(f"Maximum sequence length: {max_len}")

  for i, seq in enumerate(tqdm(data)):
    if len(seq) < max_len:
      data[i] = seq + [pad_id] * (max_len - len(seq))

  return data, max_len

In [6]:
data, max_len = padding(data)

Maximum sequence length: 20


100%|██████████| 10/10 [00:00<00:00, 54050.31it/s]


In [7]:
data

[[62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75, 0, 0, 0, 0],
 [60, 96, 51, 32, 90, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [75, 51, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [66, 88, 98, 47, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [21, 39, 10, 64, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [98, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [77,
  65,
  51,
  77,
  19,
  15,
  35,
  19,
  23,
  97,
  50,
  46,
  53,
  42,
  45,
  91,
  66,
  3,
  43,
  10],
 [70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34, 0, 0, 0, 0],
 [20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43, 0, 0, 0, 0, 0, 0]]

### Hyperparameter 세팅 및 embedding

In [8]:
d_model = 512  # model의 hidden size
num_heads = 8  # head의 개수

# d_model이 입력을 projection 시킬 임베딩 space의 차원이므로, num_heads로 나누어 떨어져야 한다.

In [9]:
embedding = nn.Embedding(vocab_size, d_model)

# B: batch size, L: maximum sequence length
batch = torch.LongTensor(data)  # (B, L)
batch_emb = embedding(batch)  # (B, L, d_model)

In [ ]:
print(batch_emb)
print(batch_emb.shape)

### Linear projection & 여러 head로 나누기

Multi-head attention 내에서 쓰이는 linear projection matrix들을 정의합니다.

In [10]:
w_q = nn.Linear(d_model, d_model)
w_k = nn.Linear(d_model, d_model)
w_v = nn.Linear(d_model, d_model)

In [11]:
w_0 = nn.Linear(d_model, d_model)

In [12]:
q = w_q(batch_emb)  # (B, L, d_model)
k = w_k(batch_emb)  # (B, L, d_model)
v = w_v(batch_emb)  # (B, L, d_model)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 20, 512])
torch.Size([10, 20, 512])
torch.Size([10, 20, 512])


Q, k, v를 `num_head`개의 차원 분할된 여러 vector로 만듭니다.

- 이론적으로는 multi-head attention을 수행하면 input을 각각 다른 head 개수만큼의 Wq, Wk, Wv로 linear transformation 해서 각각 여러번의 attention 수행한 후 concat 한 후 linear transformation 수행해준다
- 구현에서는 Wq, Wk, Wv 한 개씩
- 실제 `attention is all you need` 논문의 구현 예시는 Query vector 한개를 dim으로 쪼개서 진행한다

In [13]:
batch_size = q.shape[0]
d_k = d_model // num_heads

# num_heads * d_k로 쪼갠다
q = q.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
k = k.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
v = v.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 20, 8, 64])
torch.Size([10, 20, 8, 64])
torch.Size([10, 20, 8, 64])


In [14]:
# num_heads를 밖으로 뺌으로써
# 각 head가 (L, d_k) 만큼의 matrix를 가지고 self-attention 수행

q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 8, 20, 64])
torch.Size([10, 8, 20, 64])
torch.Size([10, 8, 20, 64])


### Scaled dot-product self-attention 구현

각 head에서 실행되는 self-attetion 과정입니다.

In [15]:
# shape - (L, L)
# 같은 sequence 내에 서로 다른 token들에게 얼마나 가중치를 두고 attention을 해야하는가
attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # (B, num_heads, L, L)
# softmax - row-wise이기 때문에 dim은 -1
attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

print(attn_dists)
print(attn_dists.shape)

tensor([[[[0.0335, 0.0580, 0.0429,  ..., 0.0567, 0.0567, 0.0567],
          [0.0499, 0.0504, 0.0512,  ..., 0.0403, 0.0403, 0.0403],
          [0.0762, 0.0323, 0.0400,  ..., 0.0275, 0.0275, 0.0275],
          ...,
          [0.0284, 0.0381, 0.0700,  ..., 0.0796, 0.0796, 0.0796],
          [0.0284, 0.0381, 0.0700,  ..., 0.0796, 0.0796, 0.0796],
          [0.0284, 0.0381, 0.0700,  ..., 0.0796, 0.0796, 0.0796]],

         [[0.0372, 0.0644, 0.0730,  ..., 0.0294, 0.0294, 0.0294],
          [0.0342, 0.0600, 0.0369,  ..., 0.0643, 0.0643, 0.0643],
          [0.0998, 0.0262, 0.0507,  ..., 0.0594, 0.0594, 0.0594],
          ...,
          [0.0438, 0.0348, 0.0384,  ..., 0.0808, 0.0808, 0.0808],
          [0.0438, 0.0348, 0.0384,  ..., 0.0808, 0.0808, 0.0808],
          [0.0438, 0.0348, 0.0384,  ..., 0.0808, 0.0808, 0.0808]],

         [[0.0370, 0.0563, 0.0400,  ..., 0.0481, 0.0481, 0.0481],
          [0.0383, 0.0723, 0.0657,  ..., 0.0256, 0.0256, 0.0256],
          [0.0684, 0.0578, 0.0374,  ..., 0

In [16]:
attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

print(attn_values.shape)

torch.Size([10, 8, 20, 64])


### 각 head의 결과물 병합

각 head의 결과물을 concat하고 동일 차원으로 linear projection합니다.

In [17]:
attn_values = attn_values.transpose(1, 2)  # (B, L, num_heads, d_k)
attn_values = attn_values.contiguous().view(batch_size, -1, d_model)  # (B, L, d_model)

print(attn_values.shape)

torch.Size([10, 20, 512])


In [18]:
# w_0 : (d_model, d_model)
# 서로 다른 의미로 foucsing 된 각 head의 self-attention 정보들을 합쳐주는 역할 수행
outputs = w_0(attn_values)

print(outputs)
print(outputs.shape)

tensor([[[ 0.0106,  0.0115, -0.0005,  ..., -0.0712, -0.1098, -0.0659],
         [ 0.0525,  0.0792, -0.0039,  ..., -0.0528, -0.0492, -0.0287],
         [ 0.0274,  0.0840, -0.0084,  ..., -0.0343, -0.0747, -0.0851],
         ...,
         [ 0.0439,  0.0649,  0.0591,  ..., -0.0244, -0.1058,  0.0321],
         [ 0.0439,  0.0649,  0.0591,  ..., -0.0244, -0.1058,  0.0321],
         [ 0.0439,  0.0649,  0.0591,  ..., -0.0244, -0.1058,  0.0321]],

        [[ 0.2657,  0.1159, -0.2594,  ..., -0.0963, -0.2405,  0.1822],
         [ 0.2875,  0.1081, -0.2328,  ..., -0.0455, -0.2609,  0.1413],
         [ 0.2616,  0.1483, -0.2944,  ..., -0.1036, -0.2274,  0.2030],
         ...,
         [ 0.3070,  0.1214, -0.2897,  ..., -0.0606, -0.2617,  0.1582],
         [ 0.3070,  0.1214, -0.2897,  ..., -0.0606, -0.2617,  0.1582],
         [ 0.3070,  0.1214, -0.2897,  ..., -0.0606, -0.2617,  0.1582]],

        [[ 0.0650,  0.0982, -0.1668,  ..., -0.0170, -0.2075,  0.0626],
         [ 0.0335,  0.1672, -0.1525,  ..., -0

## Req. 2-2 Multi-head self-attention 모듈 클래스 구현

위의 과정을 모두 합쳐 하나의 Multi-head attention 모듈 class를 구현하겠습니다.

아래 코드의 TODO 부분을 채워주세요.

In [19]:
class MultiheadAttention(nn.Module):
  def __init__(self):
    super(MultiheadAttention, self).__init__()

    # Q, K, V learnable matrices
    self.w_q = nn.Linear(d_model, d_model)
    self.w_k = nn.Linear(d_model, d_model)
    self.w_v = nn.Linear(d_model, d_model)

    # Linear projection for concatenated outputs
    self.w_0 = nn.Linear(d_model, d_model)

  # scaled-dot product attention
  def self_attention(self, q, k, v):
    attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # (B, num_heads, L, L)
    attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

    attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

    return attn_values

  def forward(self, q, k, v):
    batch_size = q.shape[0]

    # linear projection
    ################################################################################
    # TODO 1: Implement the forward pass for linear projection.                #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    q = self.w_q(q) 
    k = self.w_k(k) 
    v = self.w_v(v)  

    # head만큼 쪼개준다
    ################################################################################
    # TODO 2: Implement the forward pass for split head.                #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    q = q.view(batch_size, -1, num_heads, d_k)  
    k = k.view(batch_size, -1, num_heads, d_k) 
    v = v.view(batch_size, -1, num_heads, d_k)  

    # 각 head가 (L, d_k)의 matrix를 담당하도록 만든다
    q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
    k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
    v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

    attn_values = self.self_attention(q, k, v)  # (B, num_heads, L, d_k)
    attn_values = attn_values.transpose(1, 2).contiguous().view(batch_size, -1, d_model)  # (B, L, num_heads, d_k) => (B, L, d_model)

    return self.w_0(attn_values)

In [20]:
multihead_attn = MultiheadAttention()

outputs = multihead_attn(batch_emb, batch_emb, batch_emb)  # (B, L, d_model)

In [21]:
print(outputs)
print(outputs.shape)  # (batch_size, length, d_model)

tensor([[[ 1.3513e-02,  5.5535e-02,  3.2119e-02,  ..., -1.0572e-04,
          -6.5021e-02, -6.6141e-02],
         [ 3.5989e-02,  9.8419e-02,  8.6256e-02,  ..., -4.4619e-02,
          -1.1402e-01, -3.2597e-02],
         [ 6.4989e-02,  5.5447e-02,  6.0514e-02,  ..., -2.6617e-02,
          -8.8778e-02, -1.0302e-01],
         ...,
         [ 5.4912e-02,  3.9996e-02,  9.2343e-02,  ..., -1.2819e-02,
          -1.1423e-01, -7.3098e-02],
         [ 5.4912e-02,  3.9996e-02,  9.2343e-02,  ..., -1.2819e-02,
          -1.1423e-01, -7.3098e-02],
         [ 5.4912e-02,  3.9996e-02,  9.2343e-02,  ..., -1.2819e-02,
          -1.1423e-01, -7.3098e-02]],

        [[ 6.3359e-03,  4.4390e-01,  7.0669e-02,  ..., -2.1539e-01,
          -2.1502e-02,  1.2415e-01],
         [-3.3830e-02,  4.3597e-01,  1.0081e-01,  ..., -2.0408e-01,
          -1.4645e-02,  1.5854e-01],
         [-1.4046e-01,  4.0944e-01,  3.5762e-02,  ..., -1.3867e-01,
          -8.9681e-02,  1.4519e-01],
         ...,
         [-2.2807e-02,  4